In [71]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.api as sm
import requests
import os.path
import time
import random
import re
import math
import sklearn


from sklearn import metrics
# Standarization para la construcción de ratios (pre-built functionality under sklearn.preprocessing)
from sklearn.preprocessing import scale
# Analisis de Componentes Principales
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor

In [72]:
#DOWLOADING FILE FROM DROPBOX FIRST TIME

while not os.path.exists('EX2_DATA_BASE.xlsx'):
    time.sleep (3*random.random()); #Sleeping less than 3 seconds before going to Dropbox - avoid too many students at once.
    if not os.path.exists('EX2_DATA_BASE.xlsx'):
        print "DOWLOADING FILE EX2_DATA_BASE.xlsx FROM DROPBOX BECAUSE LOCAL FILE DOES NOT EXIST!"

        #        csvfile = urllib2.urlopen("https://dl.dropboxusercontent.com/u/28535341/EX2_DEV_DATA.xlsx")
        resp = requests.get("https://dl.dropboxusercontent.com/u/28535341/EX2_DATA_BASE.xlsx")        
        output = open('EX2_DATA_BASE.xlsx','wb')
        output.write(resp.content)
        output.close()
#DOWLOADING FILE FROM DROPBOX FIRST TIME

In [73]:
print "LOADING DATASETS..."

df = pd.read_excel(open('EX2_DATA_BASE.xlsx','rb'), sheetname='DATA')
df_dictionary = pd.read_excel(open('EX2_DATA_BASE.xlsx','rb'), sheetname='DICTIONARY')

LOADING DATASETS...


In [74]:
print "GETTING LIST OF VARIABLES..."

target_variable = str(df_dictionary[df_dictionary['TYPE_OF_VARIABLE'] == 'TARGET']['FIELD_NAME'].tolist()[0])

list_of_inputs_for_model0 = df_dictionary[df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT0']['FIELD_NAME'].tolist()

list_of_inputs_for_model1 = df_dictionary[(df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT0') | (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT1')]['FIELD_NAME'].tolist()

list_of_inputs_for_model2 = df_dictionary[(df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT0') | (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT1')| (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT2')]['FIELD_NAME'].tolist()

list_of_inputs_for_model3 = df_dictionary[(df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT0') | (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT1')| (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT2')| (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT3')]['FIELD_NAME'].tolist()

list_of_inputs_for_model4 = df_dictionary[(df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT0') | (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT1')| (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT2')| (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT3')| (df_dictionary['TYPE_OF_VARIABLE'] == 'INPUT4')]['FIELD_NAME'].tolist()

GETTING LIST OF VARIABLES...


In [75]:
list_continuous_PD=[col for col in list(df) if (col.startswith('BB_1YR_DEFAULT_PROB'))]

In [76]:
# Se incluyen solo las variables continuas

list_of_inputs_for_model0 = [ v for v in list_of_inputs_for_model0 if not (v.startswith('BEST_') |v.startswith('CUR_')
                                                                           |v.startswith('NAME')
                                                                           |v.startswith('INDUSTRY_')
                                                                           |v.startswith('TICKER'))]

list_of_inputs_for_model1 = [ v for v in list_of_inputs_for_model1 if not (v.startswith('BEST_') |v.startswith('CUR_')
                                                                           |v.startswith('NAME')
                                                                           |v.startswith('INDUSTRY_')
                                                                           |v.startswith('TICKER'))]
list_of_inputs_for_model2 = [ v for v in list_of_inputs_for_model2 if not (v.startswith('BEST_') |v.startswith('CUR_')
                                                                           |v.startswith('NAME')
                                                                           |v.startswith('INDUSTRY_')
                                                                           |v.startswith('TICKER'))]
list_of_inputs_for_model3 = [ v for v in list_of_inputs_for_model3 if not (v.startswith('BEST_') |v.startswith('CUR_')
                                                                           |v.startswith('NAME')
                                                                           |v.startswith('INDUSTRY_')
                                                                           |v.startswith('TICKER'))]
list_of_inputs_for_model4 = [ v for v in list_of_inputs_for_model4 if not (v.startswith('BEST_') |v.startswith('CUR_')
                                                                           |v.startswith('NAME')
                                                                           |v.startswith('INDUSTRY_')
                                                                           |v.startswith('TICKER'))]

In [77]:
df.shape #N<p

(116, 315)

In [78]:
# En este punto si tuviéramos un gran número de observaciones, separaríamos entre train y test.

In [79]:
# Comprobar que la Target está informada y en el rango (0,1)
df = df[(df[target_variable].between(0,1))] 

In [80]:
# Variable target y variable ID
target_variable='BB_1YR_DEFAULT_PROB_CQ4_2015'
identifier = 'TICKER'

In [81]:
df.shape

(111, 315)

In [82]:
# Tratamiento de missing value. Todas las variables son continuas por lo que se utilizn las medias.
# En el TFM habría que proceder a identificar en tipo de variables como en la práctica 1 de Gadi.
# Se procede a imputar valores sin estudiar si existen patrones. 
for var in df.columns:
    if (var in list_of_inputs_for_model4):
        missing=df[var].mean()
        df.loc[df[var].isnull(), var]=missing

In [83]:
# Tratamiento de "super outliers" 
# Se van a detectar a nivel univariante.  Un análisis más elaborado llevaría a la aplicación de técnicas multivariante. 
# 1. ¿Cómo se identifican? Criterio muy poco restrictivo si limpiamos a nivel univariante para no quedarnos sin observaciones. Optamos por un criterio simple y limpieza univariante.
# 2. Se sustituyen por el valor medio

# Outlier detection in very small sets, alternativamente MAD (mejorar para el TFM):
for var in df.columns:
    if (var in list_of_inputs_for_model4):
        media = df[var].mean()
        stdev= df[var].std()
        df.loc[abs(df[var] -media) > 3*stdev, var] = media            

In [84]:
# Construcción de ratios multiples limitado a: 
# 1. División entre todas las partidas del modelo del mismo año
# 2. Tasas de crecimiento de los ratios

In [85]:
### Modelo 0####

In [87]:
# Lista de variables para el cálculo de ratios
list0ratios = [x for x in list_of_inputs_for_model0 if x not in list_continuous_PD]
len(list0ratios)

140

In [88]:
lenls=len(list0ratios)

for i in range(0, lenls):
    v_1=df.columns[i]
    for j in range(i+1, lenls):
        v_2=df.columns[j]
        if(v_1 in list0ratios and v_2 in list0ratios):
            ratio='r_div'+v_1+v_2
            df[ratio]=np.where(df[v_2]==0,0,(df[v_1]/df[v_2]))
            list_of_inputs_for_model0.append(ratio)
            
        #tasa de crecimiento. 
            ratio='r_tasa'+v_1+v_2
            df[ratio]=np.where(df[v_2]==0,0,((df[v_1]-df[v_2])/df[v_2])) 
            list_of_inputs_for_model0.append(ratio)

In [90]:
list_of_inputs_for_model0

[u'TOTAL_EQUITY_CQ4_2013',
 u'TOTAL_EQUITY_CQ1_2014',
 u'TOTAL_EQUITY_CQ2_2014',
 u'TOTAL_EQUITY_CQ3_2014',
 u'TOTAL_EQUITY_CQ4_2014',
 u'NON_CUR_LIAB_CQ4_2013',
 u'NON_CUR_LIAB_CQ1_2014',
 u'NON_CUR_LIAB_CQ2_2014',
 u'NON_CUR_LIAB_CQ3_2014',
 u'NON_CUR_LIAB_CQ4_2014',
 u'BS_LT_BORROW_CQ4_2013',
 u'BS_LT_BORROW_CQ1_2014',
 u'BS_LT_BORROW_CQ2_2014',
 u'BS_LT_BORROW_CQ3_2014',
 u'BS_LT_BORROW_CQ4_2014',
 u'BS_CUR_LIAB_CQ4_2013',
 u'BS_CUR_LIAB_CQ1_2014',
 u'BS_CUR_LIAB_CQ2_2014',
 u'BS_CUR_LIAB_CQ3_2014',
 u'BS_CUR_LIAB_CQ4_2014',
 u'BS_ST_BORROW_CQ4_2013',
 u'BS_ST_BORROW_CQ1_2014',
 u'BS_ST_BORROW_CQ2_2014',
 u'BS_ST_BORROW_CQ3_2014',
 u'BS_ST_BORROW_CQ4_2014',
 u'BS_ACCT_PAYABLE_CQ4_2013',
 u'BS_ACCT_PAYABLE_CQ1_2014',
 u'BS_ACCT_PAYABLE_CQ2_2014',
 u'BS_ACCT_PAYABLE_CQ3_2014',
 u'BS_ACCT_PAYABLE_CQ4_2014',
 u'BS_TOT_ASSET_CQ4_2013',
 u'BS_TOT_ASSET_CQ1_2014',
 u'BS_TOT_ASSET_CQ2_2014',
 u'BS_TOT_ASSET_CQ3_2014',
 u'BS_TOT_ASSET_CQ4_2014',
 u'BS_TOT_NON_CUR_ASSET_CQ4_2013',
 u'BS_TOT_

In [91]:
df.shape # 37000 variables arggggg

(111, 6167)

In [199]:
#¿Cómo incluyo una variable que calcule tasa con un periodo anterior o cuatrimestre? para no incluir todo tipo de varaibles

In [92]:
# Reemplazamos todos los NaN por la mediana de la columna por ser ratios (las obseervacione soriginales se habían sustituido por la media)

df= df.replace([np.inf,-np.inf],0)  # Repasar si se sustituye por la media. 
#df = df.apply(lambda x: x.fillna(x.median()),axis=0)

In [95]:
# Escalar las variables para aplicación de PCA
for var in df.columns:
    if(var in list_of_inputs_for_model0):
        media=df[var].mean()
        stdev=df[var].std()
        
        df[var]=(df[var]-media) /stdev

In [97]:
# Realizadas todas las transformaciones, restringimos el df a ratios y PD para aplicar PCA

list_of_inputs_for_model0 = [ v for v in list_of_inputs_for_model0 if  (v.startswith('r') |v.startswith('BB_1YR_DEFAULT_PROB'))]

In [206]:
# Habría que hacer un split en train y test en este punto

In [111]:
#  Aplicación PCA para selección ratios que expliquen un porcentaje de la varianza ¿Deberíamos aplicar validación cruzada?

In [98]:
# Eliminar la target y el ID
dfid  = df [identifier]
dftarget = df[target_variable]

df.drop([identifier, target_variable], axis=1, inplace=True)

# Restringir el dataframe a las columnas contenidas en la lista
df_modelo0 = df[list_of_inputs_for_model0]

In [ ]:
df_modelo0

In [100]:
pca=PCA()

In [129]:
# No es necesario realizar el escalada, ya hemos normalizado
#escalado=scale(df_0)

In [102]:
# 1. Standardization (or Z-score normalization) is the process where the features are rescaled so that they’ll have the properties of a standard normal distribution with μ=0 and σ=1, where μ is the mean (average) and σ is the standard deviation from the mean.
# 2. Data Transformation
df_pca=pca.fit_transform(df_modelo0)

In [103]:
# 3. Construcción de varaible de varianza explicada
variance=pca.explained_variance_ratio_.cumsum()

In [106]:
# 4. Selección del número de componentes que expliquen un % dado de la varianza, por ejemplo 0.70
for i in range (0, len(df_modelo0.columns)):
    var_explained=variance[i:i+1]
    if var_explained >=0.70: 
        print ("Numero de Componentes", i, var_explained)
        pca=PCA(n_components=i)
        df_0pca=pca.fit_transform(df_modelo0)

('Numero de Componentes', 9, array([ 0.72543466]))
('Numero de Componentes', 10, array([ 0.74997622]))
('Numero de Componentes', 11, array([ 0.77067288]))
('Numero de Componentes', 12, array([ 0.78824274]))
('Numero de Componentes', 13, array([ 0.805526]))
('Numero de Componentes', 14, array([ 0.82152349]))
('Numero de Componentes', 15, array([ 0.83603077]))
('Numero de Componentes', 16, array([ 0.84860622]))
('Numero de Componentes', 17, array([ 0.86116996]))
('Numero de Componentes', 18, array([ 0.87197475]))
('Numero de Componentes', 19, array([ 0.88246732]))
('Numero de Componentes', 20, array([ 0.89260594]))
('Numero de Componentes', 21, array([ 0.90250249]))
('Numero de Componentes', 22, array([ 0.91058384]))
('Numero de Componentes', 23, array([ 0.91839335]))
('Numero de Componentes', 24, array([ 0.92539054]))
('Numero de Componentes', 25, array([ 0.93192558]))
('Numero de Componentes', 26, array([ 0.93836329]))
('Numero de Componentes', 27, array([ 0.94442642]))
('Numero de Com

In [32]:
# Con 10 componentes explicaríamos 99% de la varianza. No necesitamos seleccionar 110.Volveremos a aplicar PCA con 10 componentes.
# Distinto sería, por ejemplo, casos de fraude

In [107]:
pca = PCA(n_components = 10)
pca = pca.fit(df_modelo0)
df2  = pca.transform(df_modelo0)
df_pca2  = pd.DataFrame(df2,  columns=range(pca.n_components_))

In [108]:
# Repetir PCA con 10 componentes principales
df_pca2.shape

(111, 10)

In [141]:
# Convertir a un nuevo dataframe
#df_pca=pd.DataFrame(data =df_pca[::], index=list(range(0,len(df))))

In [109]:
####### Modelo 0#######
try:
    rf = RandomForestRegressor()
    rf.fit(df_pca2, dftarget)
    result=rf.fit(df_pca2, dftarget)
    y_pred = result.predict(df_pca2)
except np.linalg.linalg.LinAlgError as err:
    if 'Singular matrix' in err.message:
        print "MODEL-INVALID (Singular Matrix)"
    else:
        raise

In [110]:
print "ASSESSING THE MODEL..."

# CALCULATING PERFORMANCE ON DEVELOPMENT SAMPLE
from scipy.stats import spearmanr
spearman_correlation= spearmanr(dftarget, y_pred)
print "Spearman Rank Correlation = ", spearman_correlation

ASSESSING THE MODEL...
Spearman Rank Correlation =  SpearmanrResult(correlation=0.68899613899613887, pvalue=6.3282316166992843e-17)


In [ ]:
### Modelo 1####

In [111]:
# Lista de variables para el cálculo de ratios
list1ratios = [x for x in list_of_inputs_for_model1 if x not in list_continuous_PD]
len(list1ratios)

164

In [112]:
lenls=len(list1ratios)

for i in range(0, lenls):
    v_1=df.columns[i]
    for j in range(i+1, lenls):
        v_2=df.columns[j]
        if(v_1 in list1ratios and v_2 in list1ratios):
            ratio='r_div'+v_1+v_2
            df[ratio]=np.where(df[v_2]==0,0,(df[v_1]/df[v_2]))
            list_of_inputs_for_model1.append(ratio)
            
        #tasa de crecimiento. 
            ratio='r_tasa'+v_1+v_2
            df[ratio]=np.where(df[v_2]==0,0,((df[v_1]-df[v_2])/df[v_2])) 
            list_of_inputs_for_model1.append(ratio)